In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 8]

In [ ]:
os.listdir('../data')

## Load the data into DataFrame

In [ ]:
train_df = pd.read_csv('../data/train.csv', parse_dates = ['date'])

In [ ]:
train_df.shape

In [ ]:
train_df.head()

In [ ]:
df = train_df[(train_df['store_nbr'] == 44) & (train_df['item_nbr'] == 1503844)]

In [ ]:
store_sub = train_df[(train_df['store_nbr'] == 44) & (train_df['item_nbr'] == 1503844)]
store_sub.reset_index(drop = True, inplace = True)

In [ ]:
store_sub.shape

## View data as a function of time 

In [ ]:
ax = sns.lineplot(x = store_sub['date'], y = store_sub['unit_sales'], ci  = None)

In [ ]:
ax = sns.lineplot(x = store_sub['date'].dt.month , y = store_sub['unit_sales'], hue = store_sub['date'].dt.year, ci  = None)
ax.set_title('Sales by month')

In [ ]:
ax = sns.lineplot(x = store_sub['date'].dt.dayofweek , y = store_sub['unit_sales'], hue = store_sub['date'].dt.weekofyear, ci  = None)
ax.set_title('Sales by day of the week')

In [ ]:
ax = sns.lineplot(x = store_sub['date'].dt.day , y = store_sub['unit_sales'], hue = store_sub['date'].dt.month, ci  = None)
ax.set_title('Sales by day of month')

In [ ]:
store_sub['year'] = [d.year for d in store_sub.date]
store_sub['month'] = [d.month for d in store_sub.date]
store_sub['dayofyear'] = [d.dayofyear for d in store_sub.date]
store_sub['dayofweek'] = [d.dayofweek for d in store_sub.date]

# Draw Plot
fig, axes = plt.subplots(1, 3)
sns.boxplot(x = 'year', y = 'unit_sales', data = store_sub, ax = axes[0])
sns.boxplot(x = 'month', y = 'unit_sales', data = store_sub, ax = axes[1])
sns.boxplot(x = 'dayofweek', y = 'unit_sales', data = store_sub, ax = axes[2])
fig.tight_layout()

In [ ]:
store_sub.drop(['year', 'month', 'dayofweek', 'dayofyear'], axis = 1, inplace = True)

## Fill in the missing data 

In [ ]:
store_sub = df

In [ ]:
store_sub.head(50)

In [ ]:
# create a list of dates which contains the missing dates in the original dataframe
date_diff = store_sub['date'] - store_sub.shift(periods = 1, axis = 0)['date']

dates = [store_sub['date'].iloc[0]]
for idx, d in enumerate(date_diff[1:]):
    if d == pd.Timedelta('1 days'):
        # no break in the dates
        dates.append(store_sub['date'].iloc[idx + 1])
    else:
        # missing data
        while d > pd.Timedelta('0 days'):
            dates.append(store_sub['date'].iloc[idx + 1] - d + pd.Timedelta('1 days'))
            d -= pd.Timedelta('1 days')
            
dates = pd.DataFrame(dates, columns = ['date'])

In [ ]:
store_sub = pd.merge(dates, store_sub, how = 'outer', on = 'date')

try:
    store_sub.drop('onpromotion', axis = 1, inplace = True)
except Exception:
    pass

In [ ]:
print(store_sub.shape)
store_sub.head(60)

In [ ]:
def seasonal_mean(ts, i, n, m, w, alpha):
    """
    Compute the mean of corresponding seasonal periods
    ts: 1D array-like of the time series
    n: Seasonal window length of the time series
    """
    # weekly average
    ts_week = ts[i::-n]
    denom = w*(w**(len(ts_week) - 1) - 1)/(w-1)
    ts_week = sum(x for x in (list(map(lambda x: x[1]*(w**x[0]), [x for x in enumerate(ts_week)]))) if not np.isnan(x)) / denom
    print(ts_week)
    
    # monthly
    ts_month = ts[i::-m]
    denom = w*(w**(len(ts_month) - 1) - 1)/(w-1)
    ts_month = sum(x for x in (list(map(lambda x: x[1]*(w**x[0]), [x for x in enumerate(ts_month)]))) if not np.isnan(x)) / denom
    print(ts_month)
    print('---')
    
    avg = ((alpha * ts_week) + ((1 - alpha) * ts_month))
    return avg

for idx, row in store_sub.iterrows():
    if type(row['unit_sales']) != np.float:
        print(idx)
        store_sub.at[idx, 'unit_sales'] = seasonal_mean(store_sub['unit_sales'], idx, 7, 30, 0.7, 0.)

In [ ]:
sns.lineplot(x = 'date', y = 'unit_sales', data = store_sub)

In [ ]:
store_sub['log_unit_sales'] = np.log(store_sub['unit_sales'])

In [ ]:
sns.lineplot(x = 'date', y = 'log_unit_sales', data = store_sub)

## Holt Winters

In [13]:
from statsmodels.tsa.api import ExponentialSmoothing
import math
from sklearn.metrics import mean_squared_error

In [ ]:
train_ratio = 0.7
train, test = store_sub.iloc[:round(train_ratio * store_sub.shape[0]), :], store_sub.iloc[round(train_ratio * store_sub.shape[0]):, :]
train.reset_index(drop = True, inplace = True)
test.reset_index(drop = True, inplace = True)

print(train.shape)
print(test.shape)

In [ ]:
# for a in alpha:
#     for b in beta:
a, b, g = 0.23, 0.9, 0.9

print('a: {}, b: {}, g: {}'.format(a, b, g))
model = ExponentialSmoothing(endog = train['log_unit_sales'], trend = None, seasonal = 'add', seasonal_periods = 7)
model_fit = model.fit(smoothing_level = a, smoothing_slope = b, smoothing_seasonal = g, optimized = True)
predictions = model_fit.forecast(len(test))
test['predictions'] = np.exp(predictions.values)

error = mean_squared_error(test['unit_sales'], test['predictions'])
print('Test MSE: {}'.format(error))
print('Test RMSE: {}'.format(math.sqrt(error)))
print('\n')

In [ ]:
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
residuals.plot(kind = 'kde')
print(residuals.describe())

In [ ]:
sns.lineplot(x = 'date', y = 'unit_sales', data = test)
sns.lineplot(x = 'date', y = 'predictions', data = test)

In [ ]:
import pickle

In [ ]:
# pickle.dump(model_fit, open('holt_winters', 'wb'))

## Try the model on other items

In [2]:
items = pd.read_csv('..\data\items.csv')
df = pd.read_csv('..\\data\\train.csv')

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [4]:
# get items similar to item 1503844
items[items['item_nbr'] == 1503844]

,item_nbr,family,class,perishable
2814,1503844,PRODUCE,2022,1


In [5]:
tmp_df = df[(df['store_nbr'] == 44)]
tmp_df.shape

(3513089, 6)

In [6]:
tmp_df.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
125473872,125473872,2017-08-15,44,2123036,16.0,False
125473873,125473873,2017-08-15,44,2123727,5.0,False
125473874,125473874,2017-08-15,44,2123791,1.0,False
125473875,125473875,2017-08-15,44,2124052,12.0,True
125473876,125473876,2017-08-15,44,2127114,2.0,False


In [7]:
tmp_items = items[items['family'] == 'PRODUCE']
tmp_items.shape

(306, 4)

In [8]:
tmp_items.head()

,item_nbr,family,class,perishable
1709,1149069,PRODUCE,2070,1
2725,1473393,PRODUCE,2006,1
2726,1473394,PRODUCE,2006,1
2727,1473396,PRODUCE,2028,1
2728,1473401,PRODUCE,2008,1


In [9]:
tmp_result = tmp_df.merge(tmp_items, on='item_nbr', how='inner')

In [10]:
tmp_result.reset_index(inplace = True, drop = True)
tmp_result.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable
0,3041837,2013-03-16,44,1149069,57.0,NaN,PRODUCE,2070,1
1,3086944,2013-03-17,44,1149069,21.0,NaN,PRODUCE,2070,1
2,3129886,2013-03-18,44,1149069,1.0,NaN,PRODUCE,2070,1
3,3214178,2013-03-20,44,1149069,15.0,NaN,PRODUCE,2070,1
4,3255502,2013-03-21,44,1149069,17.0,NaN,PRODUCE,2070,1


In [16]:
rmse_list = []
for item in tmp_result['item_nbr'].unique():
    try:
        print(f'item: {item}')
        tmp = tmp_result[tmp_result['item_nbr'] == item]
        tmp['log_unit_sales'] = np.log(tmp['unit_sales'])
        print(tmp.shape)

        train_ratio = 0.7
        train, test = tmp.iloc[:round(train_ratio * tmp.shape[0]), :], tmp.iloc[round(train_ratio * tmp.shape[0]):, :]
        train.reset_index(drop = True, inplace = True)
        test.reset_index(drop = True, inplace = True)

        a, b, g = 0.23, 0.9, 0.9

        print('a: {}, b: {}, g: {}'.format(a, b, g))
        model = ExponentialSmoothing(endog = train['log_unit_sales'], trend = None, seasonal = 'add', seasonal_periods = 7)
        model_fit = model.fit(smoothing_level = a, smoothing_slope = b, smoothing_seasonal = g, optimized = True)
        predictions = model_fit.forecast(len(test))
        test['predictions'] = np.exp(predictions.values)

        error = mean_squared_error(test['unit_sales'], test['predictions'])
        print('Test MSE: {}'.format(error))
        print('Test RMSE: {}'.format(math.sqrt(error)))
        print('\n')
        rmse_list.append(math.sqrt(error))
    except Exception as e:
        print(e)
        print(item)

print(f'Total average rmse: {np.mean(rmse_list)}')

item: 1149069
(1596, 10)
a: 0.23, b: 0.9, g: 0.9
Test MSE: 929.6698521672391
Test RMSE: 30.490487896510267


item: 1473393
(1016, 10)
a: 0.23, b: 0.9, g: 0.9
Test MSE: 5179.238023205665
Test RMSE: 71.9669231189278


item: 1473394
(1016, 10)
a: 0.23, b: 0.9, g: 0.9


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3

Test MSE: 2583.868830144635
Test RMSE: 50.831769889948106


item: 1473396
(1000, 10)
a: 0.23, b: 0.9, g: 0.9
Test MSE: 245.35313115292234
Test RMSE: 15.663752141582245


item: 1473401
(958, 10)
a: 0.23, b: 0.9, g: 0.9
Test MSE: 277.7555325307345
Test RMSE: 16.665999295893855


item: 1473402
(941, 10)


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

a: 0.23, b: 0.9, g: 0.9
Test MSE: 123.65272738493175
Test RMSE: 11.119924792233613


item: 1473403
(1016, 10)
a: 0.23, b: 0.9, g: 0.9
Test MSE: 4180.536612301152
Test RMSE: 64.65706931419913


item: 1473404
(1016, 10)
a: 0.23, b: 0.9, g: 0.9
Test MSE: 5545.841089814589
Test RMSE: 74.47040412012406


item: 1473405
(1001, 10)
a: 0.23, b: 0.9, g: 0.9


...

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3

(253, 10)
a: 0.23, b: 0.9, g: 0.9
Test MSE: 123.0637654106699
Test RMSE: 11.093410900650435


item: 1696042
(5, 10)
a: 0.23, b: 0.9, g: 0.9
cannot copy sequence with size 10 to array axis with dimension 13
1696042
item: 2111870
(77, 10)
a: 0.23, b: 0.9, g: 0.9
Test MSE: 341.1208146890121
Test RMSE: 18.469456264032573


item: 2040637


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\l

(204, 10)
a: 0.23, b: 0.9, g: 0.9
Test MSE: 209.9651925781693
Test RMSE: 14.490175726269483


item: 2042210
(209, 10)
a: 0.23, b: 0.9, g: 0.9
Test MSE: 487.8748946733287
Test RMSE: 22.087890226848934


item: 2049081
(144, 10)
a: 0.23, b: 0.9, g: 0.9
Test MSE: 83.4794096045356
Test RMSE: 9.13670671547115


item: 2033554
(1, 10)
a: 0.23, b: 0.9, g: 0.9
Only 1 dimensional data supported
2033554
Total average rmse: 32.29132145977437


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
